1. Input from user
2. Local llm
3. LLM provides structured output for a function to be called
4. We call the function
5. We take the output of the function and feed that back into the llm
6. the llm provides an output for the user

In [4]:
input_prompt_from_user = """
What is Lucas profession, favorite movie and favorite book??
Use the information from the file: lucas_secrets.txt
"""

In [3]:
def get_lucas_info(file_path):
    with open(file_path, "r") as file:
        return file.read()
    
get_lucas_info(file_path="./lucas_secrets.txt")

'Lucas is an AI engineer. His favorite movie is "In Bruges" by Martin Macdonaugh\nand his favorite book is "Ethics a Nichomacea" by Aristotle.'

In [5]:
import ollama

response = ollama.chat(
    model='mistral-small3.2',
    messages=[{'role': 'user', 'content': 
        input_prompt_from_user}],

    tools=[{
      'type': 'function',
      'function': {
        'name': 'get_lucas_info',
        'description': 'Get the information about Lucas',
        'parameters': {
          'type': 'object',
          'properties': {
            'file_path': {
              'type': 'string',
              'description': 'The path to the file containing the information about Lucas',
            },
          },
          'required': ['file_path'],
        },
      },
    }],
)

print(response['message']['tool_calls'])

[{'function': {'name': 'get_lucas_info', 'arguments': {'file_path': 'lucas_secrets.txt'}}}]


In [9]:
def execute_tool_call(tool_call_output):
    
    if tool_call_output[0]['function']['name'] == 'get_lucas_info':
        return get_lucas_info(tool_call_output[0]['function']['arguments']['file_path'])
    else:
        return "Tool not found"

tool_call_output = response['message']['tool_calls']    
output_of_function =execute_tool_call(tool_call_output)
output_of_function

'Lucas is an AI engineer. His favorite movie is "In Bruges" by Martin Macdonaugh\nand his favorite book is "Ethics a Nichomacea" by Aristotle.'

In [12]:
new_input_with_more_information = f"""
The initial query was: {input_prompt_from_user}.
The output of the function was: {output_of_function}.
Provide the answer:
"""
response = ollama.chat(
    model='mistral-small3.2',
    messages=[{'role': 'user', 'content': 
        new_input_with_more_information}],
)

print(response)

{'model': 'mistral-small3.2', 'created_at': '2025-07-30T16:16:27.38742Z', 'message': {'role': 'assistant', 'content': 'Based on the information from the file `lucas_secrets.txt`, here is the answer to your query:\n\n- **Profession**: Lucas is an AI engineer.\n- **Favorite Movie**: "In Bruges" by Martin McDonagh.\n- **Favorite Book**: "Ethics: Nicomachean" by Aristotle.'}, 'done_reason': 'stop', 'done': True, 'total_duration': 4321344000, 'load_duration': 45447584, 'prompt_eval_count': 590, 'prompt_eval_duration': 933953500, 'eval_count': 68, 'eval_duration': 3340000167}


In [14]:
from IPython.display import display, Markdown

display(Markdown(f"# {output_of_function}"))

# Lucas is an AI engineer. His favorite movie is "In Bruges" by Martin Macdonaugh
and his favorite book is "Ethics a Nichomacea" by Aristotle.